In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data= pd.read_csv("scanner_data.csv", index_col=0)
data['Unit_price'] = data['Sales_Amount']/data['Quantity']
data

,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_price
1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.130000
2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.460000
3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.350000
4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.590000
5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.880000
...,...,...,...,...,...,...,...,...
131702,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333
131703,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000
131704,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000
131705,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000


In [3]:
grouped_data = data.groupby(["SKU_Category", "Date"])["Unit_price"].agg(Avg_price=lambda x: x.mean())
New_data = pd.DataFrame(grouped_data)
New_data

Avg_price
SKU_Category Date                 
01F          01/02/2016  10.449167
             01/03/2016  12.096875
             01/04/2016   4.674167
             01/06/2016  10.757222
             01/07/2016   6.359074
...                            ...
ZYU          27/11/2016   9.650000
             28/04/2016  18.035000
             28/05/2016  13.590000
             28/12/2016   8.310000
             30/11/2016  18.670000

[31507 rows x 1 columns]

In [4]:
merged_data= pd.merge(data,New_data, on=["SKU_Category","Date"], how="left")
merged_data

,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_price,Avg_price
0,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.130000,7.538889
1,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.460000,6.173333
2,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.350000,6.251087
3,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.590000,6.251087
4,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.880000,6.251087
...,...,...,...,...,...,...,...,...,...
131701,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333,5.110606
131702,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000,4.394242
131703,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000,3.360722
131704,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000,7.060000


In [5]:
sku_analysis= merged_data.groupby(["SKU"])["Unit_price"].agg(
    price_range=lambda x: x.max() - x.min(),  
    unique_prices=lambda x: x.nunique()      
).reset_index()

sku_analysis

,SKU,price_range,unique_prices
0,00GVC,0.000,1
1,00OK1,0.010,3
2,0121I,0.090,3
3,01IEO,1.670,9
4,01IQT,0.470,4
...,...,...,...
5237,ZZ2AO,0.000,1
5238,ZZDW7,11.870,2
5239,ZZM1A,0.285,6
5240,ZZNC5,0.000,1


In [6]:
# Filter based on two conditions
filtered_skus = sku_analysis.loc[(sku_analysis["price_range"] > 4) & (sku_analysis["unique_prices"] > 4), "SKU"]
filtered_sku_final = pd.DataFrame(filtered_skus)
# filtered_sku_final
final_Skus = pd.merge(sku_analysis, filtered_sku_final, on="SKU", how="inner")
final_Skus

,SKU,price_range,unique_prices
0,08UZX,6.840000,7
1,0DCAY,13.820000,22
2,0PW5H,6.870000,16
3,11H2S,12.570000,18
4,170LU,6.880000,5
...,...,...,...
102,XSAJ4,7.930000,9
103,YO9TA,4.660000,5
104,YT4PJ,4.410000,5
105,ZE55T,5.586667,8


In [7]:
# Merging both DataFrames on SKU
new_scanner_data = pd.merge(merged_data, final_Skus, on="SKU", how="inner")
new_scanner_data 

,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_price,Avg_price,price_range,unique_prices
0,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.350,6.251087,6.535000,13
1,02/01/2016,3874,15,FEW,F79YP,1.0,16.91,16.910,8.924444,4.030000,8
2,02/01/2016,9223,17,Q4N,EGB8E,1.0,10.50,10.500,13.373750,5.010000,11
3,02/01/2016,6294,19,C8Z,520UE,1.0,9.06,9.060,11.745000,5.463333,17
4,02/01/2016,2116,21,2ML,3483Q,1.0,7.29,7.290,6.173333,7.410000,8
...,...,...,...,...,...,...,...,...,...,...,...
6478,04/07/2016,11014,32834,Q4N,EGB8E,1.0,11.13,11.130,12.360583,5.010000,11
6479,04/07/2016,15910,32849,7C6,29GV1,1.0,23.29,23.290,14.070000,5.590000,5
6480,04/07/2016,21411,32875,DMA,UNS6E,2.0,21.25,10.625,10.750000,4.125000,5
6481,04/07/2016,16781,32886,J4R,ZE55T,1.0,22.29,22.290,9.232500,5.586667,8


In [58]:
new_scanner_data .to_csv("Final_data.csv", index=False)

In [59]:
new_scanner_data.columns

Index(['Date', 'Customer_ID', 'Transaction_ID', 'SKU_Category', 'SKU',
       'Quantity', 'Sales_Amount', 'Unit_price', 'Avg_price', 'price_range',
       'unique_prices'],
      dtype='object')

In [10]:
model_data = new_scanner_data[new_scanner_data["SKU"]=="EGB8E"]
model_data
#for sku in new_scanner_data[new_scanner_data["SKU"]]:
    


,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_price,Avg_price,price_range,unique_prices
2,02/01/2016,9223,17,Q4N,EGB8E,1.0,10.50,10.50,13.373750,5.01,11
98,08/01/2016,1456,1038,Q4N,EGB8E,1.0,10.50,10.50,6.678417,5.01,11
279,20/01/2016,142,3113,Q4N,EGB8E,1.0,10.50,10.50,12.381000,5.01,11
285,21/01/2016,69,3238,Q4N,EGB8E,1.0,10.50,10.50,9.127000,5.01,11
336,26/01/2016,1456,3966,Q4N,EGB8E,1.0,10.50,10.50,8.890833,5.01,11
...,...,...,...,...,...,...,...,...,...,...,...
5964,16/12/2016,19545,61962,Q4N,EGB8E,1.0,12.48,12.48,12.912000,5.01,11
6381,29/07/2016,17912,36791,Q4N,EGB8E,1.0,11.13,11.13,9.806111,5.01,11
6382,29/07/2016,15214,36797,Q4N,EGB8E,1.0,11.13,11.13,9.806111,5.01,11
6473,04/07/2016,18789,32807,Q4N,EGB8E,1.0,11.13,11.13,12.360583,5.01,11


In [11]:
model_data.describe()

,Customer_ID,Transaction_ID,Quantity,Sales_Amount,Unit_price,Avg_price,price_range,unique_prices
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,8.400000e+01,84.0
mean,12459.059524,30839.488095,1.238095,13.101310,11.133532,10.056599,5.010000e+00,11.0
std,6279.792440,18070.893397,0.651579,4.423938,1.413462,2.206363,1.787026e-15,0.0
min,69.000000,17.000000,1.000000,7.490000,7.490000,4.855714,5.010000e+00,11.0
25%,7333.000000,14709.000000,1.000000,11.130000,11.000000,8.788472,5.010000e+00,11.0
50%,15081.500000,29749.500000,1.000000,11.755000,11.130000,9.647532,5.010000e+00,11.0
75%,17597.750000,45377.500000,1.000000,12.480000,12.480000,11.684000,5.010000e+00,11.0
max,22015.000000,61962.000000,3.000000,24.970000,12.500000,15.238000,5.010000e+00,11.0


In [12]:
model_data.isna().sum()

Date              0
Customer_ID       0
Transaction_ID    0
SKU_Category      0
SKU               0
Quantity          0
Sales_Amount      0
Unit_price        0
Avg_price         0
price_range       0
unique_prices     0
dtype: int64

In [13]:
model_data.dtypes

Date               object
Customer_ID         int64
Transaction_ID      int64
SKU_Category       object
SKU                object
Quantity          float64
Sales_Amount      float64
Unit_price        float64
Avg_price         float64
price_range       float64
unique_prices       int64
dtype: object

In [14]:
# Assuming you have a DataFrame 'df' with features and target variable 'y'
# X = new_scanner_data["Unit_price"]  # Replace 'target_column' with the actual column name
# y = new_scanner_data["Quantity"]# The target variable

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = train_test_split(model_data, test_size=0.2, random_state=42)

formula= 'np.log1p(Quantity) ~ np.log1p(Unit_price)'
#building the modle on the train data
model = smf.ols(formula= formula, data=train_data, missing = 'drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:     np.log1p(Quantity)   R-squared:                       0.565
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     84.29
Date:                Thu, 16 Jan 2025   Prob (F-statistic):           2.38e-13
Time:                        10:46:45   Log-Likelihood:                 32.800
No. Observations:                  67   AIC:                            -61.60
Df Residuals:                      65   BIC:                            -57.19
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                4.1012 

In [60]:
results = {}

for sku in new_scanner_data['SKU'].unique():
    df_sku = new_scanner_data.loc[new_scanner_data['SKU'] == sku]

    # Check variability
    if df_sku['Quantity'].nunique() <= 1 or df_sku['Unit_price'].nunique() <= 1:
        continue  # Skip SKUs with insufficient variability

    # Filter invalid values
    df_sku = df_sku[(df_sku['Unit_price'] > 0) & (df_sku['Quantity'] > 0)]

    # Define independent and dependent variables
    X = np.log1p(df_sku[['Unit_price']])
    X = sm.add_constant(X)  # Add a constant term to the model
    y = np.log1p(df_sku['Quantity'])

    # Fit the linear regression model
    model = sm.OLS(y, X).fit()

    # Store results for each SKU
    results[sku] = {
         'Intercept': model.params.loc['const'],
        'params': model.params.loc['Unit_price'],  # Access by label
        'pvalues': model.pvalues.loc['Unit_price'],  # Access by label
        'rsquared': model.rsquared
    }

# Convert results to a DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df


,Intercept,params,pvalues,rsquared
CZUZX,0.706113,-0.001097,8.528332e-01,0.000056
F79YP,1.694475,-0.340636,1.874261e-06,0.058560
EGB8E,4.229698,-1.387962,7.921635e-18,0.596348
520UE,1.974133,-0.561665,9.982800e-14,0.271876
LQ555,0.685300,0.004460,7.187515e-01,0.000622
...,...,...,...,...
KSIYN,0.566401,0.052522,5.543510e-01,0.016123
FUHQ1,2.127473,-0.453861,4.472728e-01,0.099229
JH7HA,2.141502,-0.443159,3.842001e-01,0.128012
170LU,1.939484,-0.457488,7.940658e-02,0.217893


In [61]:
#filtering the regression coeffs which has low p-values and rsquared greater than 0.40
results_df_final= results_df[(results_df["pvalues"]<0.05) & (results_df["rsquared"]>0.40)]

results_df_final.index.name= "SKU"
results_df_final

,Intercept,params,pvalues,rsquared
SKU,,,,
EGB8E,4.229698,-1.387962,7.921635e-18,0.596348
UCK6F,2.475528,-0.737922,2.209446e-38,0.648432
LKDTY,2.454907,-0.771022,1.326980e-09,0.444632
0PW5H,3.626075,-1.150938,1.059054e-19,0.465922
6SPWZ,3.697421,-1.283279,7.597398e-05,0.684900
B82LD,2.400220,-0.682571,6.930926e-05,0.449364
MLAK3,1.098556,-0.154701,1.011038e-15,0.993764
7Q5A4,1.846960,-0.397566,3.392593e-04,0.464938
H15DO,3.511312,-0.812645,2.418721e-06,0.455867


In [62]:
results_df_final.to_csv("coeffs.csv")

## APPENDIX

In [ ]:
#choosing single product
trans_count= new_scanner_data.groupby(["SKU"])["Transaction_ID"].count()
transaction= trans_count.nlargest(10)
top_transaction= pd.DataFrame(transaction)
top_transaction

In [ ]:
merged_data=pd.merge(new_scanner_data,top_transaction, how='inner')
merged_data

In [ ]:

final_data = new_scanner_data[new_scanner_data["SKU"] == "F79YP"]
final_data

In [ ]:
final_data.price_range.value_counts()